In [1]:
# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
# make variables display whenever they are on their own line (not just the last line of a cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from utils import *

In [8]:
tf.reset_default_graph()
sess = tf.Session()
input_ph = tf.placeholder('float', (None, 224, 224, 3))

In [30]:
dataset = make_dataset_from_tfrecord(['conv1.tfrecord'], [224, 224, 64])

In [34]:
sess.run(dataset.make_one_shot_iterator().get_next()).shape

(224, 224, 64)

In [10]:
make_encoder(input_ph, sess)
reconstructed_image = make_decoder(vgg[1]['vgg_19/conv2/conv2_1'], [(3, 64), 'upsample', (3, 64), (3, 3)], sess)

INFO:tensorflow:Restoring parameters from vgg_19.ckpt


(<tf.Tensor 'vgg_19/fc8/squeezed:0' shape=(?, 1000) dtype=float32>,
 OrderedDict([('vgg_19/conv1/conv1_1',
               <tf.Tensor 'vgg_19/conv1/conv1_1/Relu:0' shape=(?, 224, 224, 64) dtype=float32>),
              ('vgg_19/conv1/conv1_2',
               <tf.Tensor 'vgg_19/conv1/conv1_2/Relu:0' shape=(?, 224, 224, 64) dtype=float32>),
              ('vgg_19/pool1',
               <tf.Tensor 'vgg_19/pool1/MaxPool:0' shape=(?, 112, 112, 64) dtype=float32>),
              ('vgg_19/conv2/conv2_1',
               <tf.Tensor 'vgg_19/conv2/conv2_1/Relu:0' shape=(?, 112, 112, 128) dtype=float32>),
              ('vgg_19/conv2/conv2_2',
               <tf.Tensor 'vgg_19/conv2/conv2_2/Relu:0' shape=(?, 112, 112, 128) dtype=float32>),
              ('vgg_19/pool2',
               <tf.Tensor 'vgg_19/pool2/MaxPool:0' shape=(?, 56, 56, 128) dtype=float32>),
              ('vgg_19/conv3/conv3_1',
               <tf.Tensor 'vgg_19/conv3/conv3_1/Relu:0' shape=(?, 56, 56, 256) dtype=float32>),
      

NameError: name 'vgg' is not defined

In [ ]:
loss = tf.sqrt(tf.reduce_mean(tf.square(input_ph - reconstructed_image)))

In [ ]:
train(loss, dataset, input_ph, sess, num_epochs=10, lr=1e-4)

In [ ]:
it = dataset.make_one_shot_iterator()

In [ ]:
img = sess.run(tf.cast(it.get_next(), tf.uint8))

In [ ]:
plt.imshow(img)

In [ ]:
decoded_img = sess.run(reconstructed_image, feed_dict={input_ph: img[np.newaxis,:,:,:]})

In [ ]:
decoded_img = np.squeeze(decoded_img)

In [ ]:
decoded_img[decoded_img > 255] = 255

In [ ]:
plt.imshow(decoded_img.astype(np.uint8))

In [11]:
writer = tf.summary.FileWriter('summaries', sess.graph)

In [18]:
compute_features('../val2017', 'conv1.tfrecord', out_layer_name='vgg_19/conv1/conv1_1', num_images=200)

INFO:tensorflow:Restoring parameters from vgg_19.ckpt
